# NESTML active dendrite tutorial

In this tutorial, we will create a neuron model with an active dendritic compartment, that will fire a dendritic spike if the total synaptic current exceeds a threshold.


In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import nest
import numpy as np
import os
import uuid
from pynestml.frontend.pynestml_frontend import to_nest, install_nest

NEST_SIMULATOR_INSTALL_LOCATION = "/home/beeblebrox/nest-simulator-build"

ModuleNotFoundError: No module named 'nest'

In [ ]:
nestml_active_dend_model = '''
neuron iaf_psc_exp_nonlineardendrite:
  initial_values:
    V_m mV = 0 mV         # membrane potential
    t_dend_sp ms = 0 ms   # dendritic spike timer
    dend_curr_enabled real = 1.   # set to 1 to allow synaptic dendritic currents to contribute to V_m integration, 0 otherwise
    I_dend_spike pA = 0 pA
  end

  equations:
    kernel I_kernel1 = exp(-t / tau_syn1)
    kernel I_kernel2 = (e / tau_syn2) * t * exp(-t / tau_syn2)
    kernel I_kernel3 = exp(-t / tau_syn3)

    recordable inline I_dend pA = convolve(I_kernel2, I_2)

    inline I_syn pA = convolve(I_kernel1, I_1) + dend_curr_enabled * I_dend + I_dend_spike + convolve(I_kernel3, I_3) + I_e

    V_m' = -(V_m - E_L) / tau_m + I_syn / C_m
  end

  parameters:
    C_m pF = 250 pF          # capacity of the membrane
    tau_m ms = 20 ms         # membrane time constant
    tau_syn1 ms = 10 ms      # time constant of synaptic current, port 1
    tau_syn2 ms = 10 ms      # time constant of synaptic current, port 2
    tau_syn3 ms = 10 ms      # time constant of synaptic current, port 3
    V_th mV = 25 mV          # spike threshold
    V_reset mV = 0 mV        # reset voltage
    I_e    pA = 0 pA         # external current
    E_L    mV = 0 mV         # resting potential

    # dendritic action potential
    i_th pA = 60 pA          # current-threshold for a dendritic spike
    i_dend_sp pA = 150 pA    # current clamp value for I_dend during a dendritic spike
    T_dend_sp ms = 10 ms     # time window over which the dendritic current clamp is active
  end

  input:
    I_1 pA <- spike
    I_2 pA <- spike
    I_3 pA <- spike
  end

  output: spike

  update:
    # solve ODEs
    integrate_odes()

    if t_dend_sp > 0 ms:
      t_dend_sp -= resolution()
      if t_dend_sp <= 0 ms:
        t_dend_sp = 0 ms
        dend_curr_enabled = 1.
        I_dend = 0 pA
        I_dend' = 0 pA/ms
        I_dend_spike = 0 pA
      end
    end

    if I_dend > i_th:
      # current-threshold, emit a dendritic spike
      dend_curr_enabled = 0.
      t_dend_sp = T_dend_sp
      I_dend_spike = i_dend_sp
    end

    # emit somatic spike
    if V_m > V_th:
      emit_spike()
      V_m = V_reset
    end
  end
end
'''

Save to a temporary file and make the model available to instantiate in NEST:

In [ ]:
with open("iaf_psc_exp_nonlineardendrite.nestml", "w") as nestml_model_file:
    print(nestml_active_dend_model, file=nestml_model_file)

to_nest(input_path="iaf_psc_exp_nonlineardendrite.nestml",
        target_path="/tmp/nestml-active-dend-target",
        module_name="nestml_active_dend_module",
        suffix="_nestml",
        logging_level="INFO")
install_nest("/tmp/nestml-active-dend-target", NEST_SIMULATOR_INSTALL_LOCATION)
nest.Install("nestml_active_dend_module")

### Running the simulation in NEST

Let's define a function that will instantiate the active dendrite model, run a simulation, and plot and return the results.

In [2]:
def evaluate_neuron(neuron_name, neuron_parms=None, I_e=0.,
                    mu=0., sigma=0., t_sim=300., plot=True):
    """
    Run a simulation in NEST for the specified neuron. Inject a stepwise
    current and plot the membrane potential dynamics and spikes generated.
    """
    dt = .1   # [ms]

    MAX_SSE = 1E-12

    I_dend_alias_name = 'I_dend'


    neuron_name = iaf_psc_exp_nonlineardendrite_nestml


    nest.ResetKernel()
    try:
        nest.Install("nestml_active_dend_module")
    except :
        pass
    neuron = nest.Create(neuron_name)
    if neuron_parms:
        for k, v in neuron_parms.items():
            nest.SetStatus(neuron, k, v)

    sg = nest.Create("spike_generator", params={"spike_times": [10., 20., 30.]})
    nest.Connect(sg, neuron, syn_spec={"receptor_type": 2, "weight": 30., "delay": 1.})
    
    multimeter = nest.Create("multimeter")
    multimeter.set({"record_from": ["V_m", I_dend_alias_name],
                    "interval": dt})
    sr = nest.Create("spike_recorder")

    nest.Connect(sg, neuron)
    nest.Connect(multimeter, neuron)
    nest.Connect(neuron, sr)
    
    nest.Simulate(t_sim)

    dmm = nest.GetStatus(multimeter)[0]
    Voltages = dmm["events"]["V_m"]
    tv = dmm["events"]["times"]
    dSD = nest.GetStatus(sr, keys='events')[0]
    spikes = dSD['senders']
    ts = dSD["times"]
    
    _idx = [np.argmin((tv - spike_time)**2) - 1 for spike_time in ts]
    V_m_at_spike_times = Voltages[_idx]
    timevec = mm.get("events")["times"]
    I_dend_alias_ts = mm.get("events")[I_dend_alias_name]

    if plot:
        fig, ax = plt.subplots(3, 1)
        ax[0].plot(timevec, I_dend_alias_ts, label="I_dend")
        ax[2].plot(timevec, mm.get("events")["V_m"], label="V_m")
        for _ax in ax:
            _ax.legend()
            _ax.grid()

    return ts

In [3]:
evaluate_neuron("iaf_psc_exp_nonlineardendrite_nestml")

NameError: name 'iaf_psc_exp_nonlineardendrite_nestml' is not defined